In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

# %matplotlib inline
import pandas as pd

from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.initial_stock_loss = 10
        self.position_size = 2
        self.last_order = ''
        self.data = self.datas[0].open
        self.sma1 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=10
        )
        self.sma2 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=20
        )
        self.report = []
        self.last_order_data = {}
        
    def calc_qty(self):
        cash = cerebro.broker.cash
        position_val = (cash * self.position_size) / 100
        
        stock_risk_val = (self.initial_stock_loss * self.datas[0].open) / 100
        qty = position_val / stock_risk_val
        return qty
    
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.data[0])
        
        if self.sma1[0] > self.sma2[0]:
            if not self.last_order or self.last_order == 'sell':
                self.buy(size=self.calc_qty())
                self.log('Buy signal')
                self.last_order = 'buy'
                self.last_order_data = {
                    'start': self.datas[0].datetime.date(0),
                    'start_price': self.data[0]
                }
        
        elif self.sma1[0] < self.sma2[0]:
            if self.last_order == 'buy':
                self.sell(size=self.calc_qty())
                self.log('Sell signal')
                self.last_order_data['end'] = self.datas[0].datetime.date(0)
                self.last_order_data['end_price'] = self.data[0]
                commission = (self.data[0] * 0.001) / 100) * self.
                self.last_order_data['commission'] = commission
                profit = self.data[0] - self.last_order_data['start_price']
                self.last_order_data['profit'] = profit
                self.last_order_data['final profit'] = profit - commission
                self.last_order = 'sell'
                self.report.append(self.last_order_data)
    
    def stop(self):
        df = pd.DataFrame(self.report)
        df.to_csv('/tmp/profit.csv')
                


# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
datapath = os.path.join('data.csv')

# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
    dataname=datapath,
    # Do not pass values before this date
    fromdate=datetime.datetime(2000, 1, 1),
    # Do not pass values before this date
    todate=datetime.datetime(2000, 12, 31),
    # Do not pass values after this date
    reverse=False
)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()
print(cerebro.datas)
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
b = Bokeh(plot_mode='single', scheme=Tradimo(), plot_width=750, plot_height=200)

cerebro.plot(b)

/home/hitul/.pyenv/versions/3.6.3/envs/iit/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/hitul/.pyenv/versions/3.6.3/envs/iit/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

Starting Portfolio Value: 100000.00
Final Portfolio Value: 98567.47


[[<backtrader_plotting.bokeh.bokeh.FigurePage at 0x7f16f1a9a6a0>]]